# Tutorial 1.8: Prompt Optimization with GEPA

![](images/9_Prompt-Optimization-with-GEPA.png)

## Automatically Improve Prompts Using MLflow's GEPA Integration

In Tutorial 1.5, we manually iterated on prompts — writing better versions by hand and versioning them in the Prompt Registry. But what if an algorithm could do this automatically?

This notebook demonstrates **GEPA (Genetic-Pareto)**, an automatic prompt optimization algorithm integrated into MLflow via `mlflow.genai.optimize_prompts()`.

### What You'll Learn

- How GEPA automatically improves prompts
- Using `mlflow.genai.optimize_prompts()` with the Prompt Registry
- Evaluating prompt quality with `Correctness` scorer
- Comparing original vs. optimized prompts

### Prerequisites
- Completed Notebook 1.5 (Prompt Management) and 1.7 (Evaluating Agents)
- Understanding of the Prompt Registry and evaluation scorers

### Estimated Time: 10-15 minutes

---
## Step 1: How GEPA Works

**GEPA (Genetic-Pareto)** optimizes prompts through an iterative cycle:

```
1. EVALUATE  →  Run the prompt on training examples, score with a judge
2. REFLECT   →  Use an LLM to analyze failures and propose improvements
3. MUTATE    →  Generate improved prompt variations
4. SELECT    →  Keep the best-performing candidates (Pareto-optimal)
5. REPEAT    →  Continue until budget exhausted or convergence
```

### Manual vs. Automatic Optimization

| Approach | Method | Effort | Consistency |
|----------|--------|--------|-------------|
| **Manual** (Notebook 1.5) | Human writes better prompts | High | Variable |
| **GEPA** (This notebook) | Algorithm evolves prompts | Low | Systematic |

### Integration with Prompt Registry

GEPA works directly with MLflow's Prompt Registry:
- **Reads** your registered prompt as the starting point
- **Optimizes** it through the evaluate-reflect-mutate cycle
- **Registers** the improved version automatically as a new version

> **Note:** GEPA requires the `gepa` package. Install it with: `pip install gepa`

---
## Step 2: Environment Setup

In [1]:
import mlflow
from dotenv import load_dotenv
from utils.clnt_utils import is_databricks_ai_gateway_client, get_databricks_ai_gateway_client, get_openai_client, get_ai_gateway_model_names

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("09-prompt-optimization")

# Configure client and model based on provider
use_databricks_provider = is_databricks_ai_gateway_client()
if use_databricks_provider:
    client = get_databricks_ai_gateway_client()
    model_name = get_ai_gateway_model_names()[0]
    optimizer_model = f"databricks:/{model_name}"
else:
    client = get_openai_client()
    model_name = "gpt-5.2"
    optimizer_model = f"openai:/{model_name}"

# Enable autologging
mlflow.openai.autolog()

print("\u2705 Environment configured")
print(f"   Provider: {'Databricks AI Gateway' if use_databricks_provider else 'OpenAI'}")
print(f"   Model: {model_name}")
print(f"   Optimizer model: {optimizer_model}")
print(f"   Tracking URI: {mlflow.get_tracking_uri()}")

2026/02/09 14:38:46 INFO mlflow.tracking.fluent: Experiment with name '09-prompt-optimization' does not exist. Creating a new experiment.


✅ Environment configured
   Provider: OpenAI
   Model: gpt-5.2
   Optimizer model: openai:/gpt-5.2
   Tracking URI: http://localhost:5000


---
## Step 3: Register a Baseline Prompt

We'll use the same basic Q&A prompt from Notebook 1.5's Prompt Library (`qa_simple`). We register it fresh here so this notebook is self-contained.

This minimal prompt is an ideal optimization target — it has maximum room for GEPA to improve it.

In [2]:
# Register the baseline prompt (same template as qa_simple from Notebook 1.5)
baseline_prompt = mlflow.genai.register_prompt(
    name="gepa-qa-simple",
    template="Answer this question: {{ question }}",
    commit_message="Baseline prompt for GEPA optimization",
    tags={"author": "jules", "use_case": "Simple Q&A", "status": "baseline"}
)

print("\u2705 Baseline prompt registered")
print(f"   Name: {baseline_prompt.name}")
print(f"   Version: {baseline_prompt.version}")
print(f"   URI: {baseline_prompt.uri}")
print(f"   Template: '{baseline_prompt.template}'")

2026/02/09 14:38:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: gepa-qa-simple, version 1


✅ Baseline prompt registered
   Name: gepa-qa-simple
   Version: 1
   URI: prompts:/gepa-qa-simple/1
   Template: 'Answer this question: {{ question }}'


---
## Step 4: Prepare Training Data and Predict Function

GEPA needs two things:
1. **Training data** — example input/output pairs so it can evaluate prompt quality
2. **Predict function** — a callable that loads the prompt, fills it, and calls the LLM

### Why Training Data Design Matters

GEPA improves prompts by finding gaps between actual outputs and expected responses.
If the baseline prompt already produces near-perfect answers (common with powerful LLMs),
GEPA has **no signal to improve** and will register the original template unchanged.

To give GEPA room to work, our training examples require **specific structure and detail**
that the bare-bones prompt `"Answer this question: ..."` won't naturally produce.

In [3]:
from mlflow.genai import optimize_prompts
from mlflow.genai.optimize.optimizers import GepaPromptOptimizer
from mlflow.genai.scorers import Correctness

# Training data: questions that require STRUCTURED, DETAILED responses.
# The bare-bones baseline prompt won't guide the LLM to produce these formats,
# giving GEPA a clear signal to add structure/instructions to the prompt.
train_data = [
    {
        "inputs": {"question": "Explain MLflow Tracking to a beginner in exactly 3 sentences."},
        "expectations": {
            "expected_response": (
                "MLflow Tracking is a component that automatically logs your machine learning experiments. "
                "It records parameters, metrics, and model artifacts so you can compare different runs side by side. "
                "You can view all your experiments through the MLflow UI dashboard."
            )
        },
    },
    {
        "inputs": {"question": "List the top 3 benefits of using vector embeddings. Number each benefit."},
        "expectations": {
            "expected_response": (
                "1. Semantic similarity — embeddings capture meaning, so related concepts like 'happy' and 'joyful' are close in vector space.\n"
                "2. Efficiency — they compress high-dimensional sparse data into compact dense vectors, making computation faster.\n"
                "3. Transfer learning — pre-trained embeddings can be fine-tuned for specific downstream tasks without training from scratch."
            )
        },
    },
    {
        "inputs": {"question": "Compare RAG and fine-tuning in exactly 2 sentences."},
        "expectations": {
            "expected_response": (
                "RAG retrieves relevant documents at inference time to augment the LLM's context, "
                "while fine-tuning modifies the model's weights on domain-specific training data. "
                "RAG allows dynamic knowledge updates without retraining, while fine-tuning creates a "
                "specialized model that may lose some general capabilities."
            )
        },
    },
    {
        "inputs": {"question": "What is prompt engineering? Answer with a definition followed by 2 best practices."},
        "expectations": {
            "expected_response": (
                "Prompt engineering is the practice of designing and refining instructions given to LLMs "
                "to produce desired outputs reliably.\n\n"
                "Best practices:\n"
                "1. Be specific and explicit — clearly state the format, length, and style you expect.\n"
                "2. Provide examples — include one or two input/output examples to guide the model's behavior."
            )
        },
    },
    {
        "inputs": {"question": "Describe the MLflow Model Registry in one paragraph of 3-4 sentences."},
        "expectations": {
            "expected_response": (
                "The MLflow Model Registry is a centralized store for managing the full lifecycle of ML models. "
                "It provides model versioning, so you can track how models evolve over time. "
                "Teams can transition models through stages like Staging and Production using aliases. "
                "It also supports annotations and approval workflows for collaborative model governance."
            )
        },
    },
    {
        "inputs": {"question": "What is a vector database? Structure your answer as: Definition, then Use Cases (2 bullet points)."},
        "expectations": {
            "expected_response": (
                "A vector database is a specialized database designed to store, index, and efficiently query "
                "high-dimensional vector embeddings.\n\n"
                "Use cases:\n"
                "• Semantic search — finding documents or products by meaning rather than keyword matching.\n"
                "• RAG pipelines — retrieving relevant context for LLM generation to improve answer accuracy."
            )
        },
    },
]


# Predict function: GEPA calls this repeatedly during optimization.
# During optimization, GEPA patches PromptVersion.template so that
# load_prompt() returns the MUTATED template instead of the original.
def predict_qa(question: str) -> str:
    """Load the prompt from the registry, fill it, and call the LLM."""
    prompt = mlflow.genai.load_prompt(baseline_prompt.uri)
    filled = prompt.format(question=question)

    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": filled}],
        temperature=0.3,
    )
    return response.choices[0].message.content


print(f"\u2705 Training data prepared: {len(train_data)} examples")
print("   (Questions require specific structure/format the baseline prompt can't guide)")
print("\u2705 Predict function defined")
print(f"   Loads prompt from: {baseline_prompt.uri}")

✅ Training data prepared: 6 examples
   (Questions require specific structure/format the baseline prompt can't guide)
✅ Predict function defined
   Loads prompt from: prompts:/gepa-qa-simple/1


---
## Step 5: Run GEPA Optimization

Now we run the optimization. GEPA will:
1. Evaluate the baseline prompt using the `Correctness` scorer
2. Reflect on failures and generate improved variations
3. Select the best candidates and repeat
4. Register the optimized prompt as a new version in the Prompt Registry

> **Note:** This may take 3-5 minutes. The training examples require structured responses
> that the bare-bones prompt can't produce well, giving GEPA a clear optimization signal.

In [4]:
import logging
from ipykernel.iostream import OutStream

# === Fix for GEPA Unicode surrogate characters ===
# GEPA's internal output contains Unicode surrogates that crash Jupyter's
# ZMQ/tornado JSON encoder. We fix at two levels:
#
# Level 1: Patch OutStream.write at the CLASS level to sanitize all output.
# This ensures surrogates are stripped before they reach ipykernel's buffer,
# regardless of how write() is called (instance, class, or thread).
_orig_outstream_write = OutStream.write

def _safe_outstream_write(self, string):
    if isinstance(string, str):
        string = string.encode("utf-8", errors="replace").decode("utf-8")
    return _orig_outstream_write(self, string)

OutStream.write = _safe_outstream_write

# Level 2: Suppress tornado/ZMQ error log messages for any surrogates
# that reach non-stdout kernel messages (these errors are non-fatal noise).
for _logger_name in ("tornado.general", "tornado.application"):
    logging.getLogger(_logger_name).setLevel(logging.CRITICAL)

# Run GEPA prompt optimization
print("\ud83d\udd04 Running GEPA prompt optimization...\n")
print("   This will iterate through evaluate \u2192 reflect \u2192 mutate \u2192 select cycles.")
print("   Budget: 100 metric calls (may take 3-5 minutes)\n")

result = optimize_prompts(
    predict_fn=predict_qa,
    train_data=train_data,
    prompt_uris=[baseline_prompt.uri],
    optimizer=GepaPromptOptimizer(
        reflection_model=optimizer_model,
        max_metric_calls=100,
        display_progress_bar=False,
    ),
    scorers=[Correctness(model=optimizer_model)],
)

print("\n\u2705 GEPA optimization complete!")

2026/02/09 14:39:09 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2026/02/09 14:39:09 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.
/Users/jules/git-repos/mlflow-misc/.venv/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(


Iteration 0: Base program full valset score: 0.3333333333333333 over 6 / 6 examples
Iteration 1: Selected program 0 score: 0.3333333333333333
Iteration 1: Proposed new text for gepa-qa-simple: Answer the user’s question directly and follow any formatting constraints in the question (e.g., “3–4 sentences,” “number each benefit,” “Definition then 2 bullet points”).

Important: Only state information that is explicitly supported by the provided/reference material for this task. Do NOT add plausible-sounding extras or unstated features. Avoid introducing domain-specific terms unless the source text mentions them (e.g., don’t claim MLflow Model Registry uses “aliases” for stage transitions or that it supports “annotations” unless that is explicitly stated; don’t mention “transfer learning,” “pre-trained embeddings,” “fine-tuning,” or “compression of sparse vectors into dense vectors” unless explicitly supported).

When describing concepts, prefer safe, broadly supported statements such as:


2026/02/09 14:43:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: gepa-qa-simple, version 2


Iteration 17: New subsample score 0.0 is not better than old score 1.0, skipping
🏃 View run unleashed-trout-623 at: http://localhost:5000/#/experiments/1/runs/bf28b219f44443e998c6ccd88197d48e
🧪 View experiment at: http://localhost:5000/#/experiments/1

✅ GEPA optimization complete!


[Trace(trace_id=tr-2e743fb0681b38beb21b07c79b1363ab), Trace(trace_id=tr-3f74f64dea3f9a6116b2acf584fc38b7), Trace(trace_id=tr-1281feb02a4280214b53e4c78304789b), Trace(trace_id=tr-6d4cf26280967fb25eb9c25d4b9e5e9e), Trace(trace_id=tr-21e749968bb7cf56a80d5d42a74de86d), Trace(trace_id=tr-9b9b33a8deeb08e97e514b976ec8309e), Trace(trace_id=tr-8a7dc2973914e6772fb32c59cef70983), Trace(trace_id=tr-48b197bda70063eeb76f6dd589ea99ac), Trace(trace_id=tr-8b8f48e5fbd8ff0983ee394508a2cb96), Trace(trace_id=tr-baee490042e6b64c9dbe316dc20001c7)]

---
## Step 6: Compare Original vs. Optimized Prompt

In [5]:
def _safe(s):
    """Strip Unicode surrogates for safe display in Jupyter."""
    if isinstance(s, str):
        return s.encode("utf-8", errors="replace").decode("utf-8")
    return str(s)

# Display before/after comparison
print("=" * 70)
print("\ud83d\udcca GEPA Optimization Results")
print("=" * 70)

print("\n\ud83d\udcc8 Score Improvement:")
if result.initial_eval_score is not None:
    print(f"   Initial score: {result.initial_eval_score:.3f}")
else:
    print("   Initial score: N/A")
if result.final_eval_score is not None:
    print(f"   Final score:   {result.final_eval_score:.3f}")
else:
    print("   Final score:   N/A")
if result.initial_eval_score is not None and result.final_eval_score is not None:
    improvement = result.final_eval_score - result.initial_eval_score
    print(f"   Improvement:   {improvement:+.3f}")

# Load the optimized prompt directly from the registry to ensure we
# see the actual registered version (not just the in-memory object)
optimized = result.optimized_prompts[0]
registry_prompt = mlflow.genai.load_prompt(f"prompts:/{optimized.name}/{optimized.version}")

print(f"\n\ud83d\udcdd Original Prompt (version {baseline_prompt.version}):")
print(f"   '{baseline_prompt.template}'")

print(f"\n\ud83d\ude80 Optimized Prompt (version {optimized.version}):")
print(f"   '{_safe(registry_prompt.template)}'")

if baseline_prompt.template.strip() == _safe(registry_prompt.template).strip():
    print("\n\u26a0\ufe0f  Note: The optimized template is identical to the baseline.")
    print("   This can happen when the baseline already scores well on the")
    print("   training data. Try adding harder examples or increasing the budget.")

print("\n\ud83d\udd17 The optimized prompt has been automatically registered")
print(f"   as version {optimized.version} in the Prompt Registry!")
print(f"   View it in MLflow UI \u2192 Prompt Registry \u2192 {_safe(optimized.name)}")

print("\n" + "=" * 70)
print("\n\ud83d\udca1 Key Takeaway:")
print("   GEPA automatically learned to add structure, instructions,")
print("   and constraints that we would normally write by hand.")
print("   Combined with the Prompt Registry, optimized prompts are")
print("   versioned and ready for deployment via aliases.")

Exception in callback BaseAsyncIOLoop._handle_events(43, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(43, 1)>
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 71-72: surrogates not allowed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/jules/git-repos/mlflow-misc/.venv/lib/python3.11/site-packages/jupyter_client/session.py", line 143, in orjson_packer
    return orjson.dumps(obj, default=json_default, option=option)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: str is not valid UTF-8: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jules/git-repos/mlflow-misc/.venv/lib/python3.11/site-packages/jupyter_client/session.py", line 103, in json_packer
    ).encode("utf8", errors="surrogateescape")
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'utf-8

---
## Summary

In this notebook, you learned:

1. How **GEPA** automatically optimizes prompts through evaluate-reflect-mutate cycles
2. Using `mlflow.genai.optimize_prompts()` with the **Prompt Registry**
3. Preparing **training data** and a **predict function** for optimization
4. Comparing **before/after** prompt quality with the `Correctness` scorer
5. Optimized prompts are **automatically versioned** in the registry

### Key Takeaways

- **Automate what you can**: GEPA systematically improves prompts that would take many manual iterations
- **Data-driven optimization**: Training examples define what "good" looks like for the algorithm
- **Registry integration**: Optimized prompts flow directly into the Prompt Registry for versioning and deployment
- **Combine approaches**: Use GEPA for initial optimization, then fine-tune manually if needed

### What's Next?

**\ud83d\udcd3 Notebook 1.9: Complete RAG Application**

Learn how to:
- Build a full RAG pipeline with end-to-end tracing
- Evaluate RAG quality with RAGAS metrics
- Track performance, cost, and retrieval quality